In [1]:
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup 
import time
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_df_users = pd.DataFrame(columns = ['제목','링크','평점','유저 이름'])
data_df = pd.DataFrame(columns = ['제목','링크','평점','유저 이름'])

service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

driver.get("https://pedia.watcha.com/ko-KR/staffmades/267") 
time.sleep(3)

# 아래 코드가 특정 웹페이지에서 Selenium을 활용해 스크롤 다운하는 코드입니다.
# 가장 아래까지 스크롤 다운 후 매번 0.8초씩 로딩을 기다린 후 스크롤 다운을 마지막까지 진행합니다.
while True:
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
         break

# 광고 지우기
close_button = '/html/body/div/div/div[2]/div/div/div/button/span'
driver.find_element_by_xpath(close_button).click()

# Beautifulsoup으로 영화 데이터
movie_name = []
url = driver.current_url
movie_url = 'https://pedia.watcha.com'
movie_link = []
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
movies = soup.find_all('li',{'class':'css-8y23cj'})

for movie in movies:
    movie_name.append(movie.find('a').get('title'))
    movie_link.append(movie_url + movie.find('a').get('href'))

# 드라이버 끝내기
driver.close()
driver.quit()

df_movie = pd.DataFrame(zip(movie_name,movie_link),columns=['왓챠피디아 평점 TOP 영화','링크'])

df_movie

In [5]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

# 영화 각각 링크로 들어가기
for i in range (0, 299):
    driver.get(df_movie['링크'][0]) #여기서 반복문
    time.sleep(2)

    # 광고 지우기
    close_button = '/html/body/div/div/div[2]/div/div/div/button/span'
    driver.find_element_by_xpath(close_button).click()
    
    # 코멘트 더보기 클릭
    time.sleep(SCROLL_PAUSE)
    comment_button_xpath = '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div/div/div[1]/div[1]/div/div/section[5]/div[1]/div/header/div/div/a'
    driver.find_element_by_xpath(comment_button_xpath).click()
    
    # 스크롤 내리기 (4초; 생각보다 코멘트 로딩이 느림 / 스크롤이 너무 빨리 내려가면 더이상 안 내려가는 오류발생)
    while True:
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
         break
    
    # Beautifulsoup으로 유저 데이터
    user_name = []
    review_url = driver.current_url
    movie_url = 'https://pedia.watcha.com'
    user_link = []
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    users = soup.find_all('div',{'class':'css-1cvf9dk'})

    for user in users:
        user_name.append(user.find('a').get('title'))
        user_link.append(movie_url + user.find('a').get('href'))

    df_user = pd.DataFrame(zip(user_name,user_link),columns=['유저','유저 링크'])
    
    # 유저 데이터 행 구하기; 링크 index용
    num_user = df_user.shape[0]
    
    # 유저 링크 하나씩 들어가기
    for j in range (1, num_user):
        user_page = df_user['유저 링크'][j]
        user_movie = ''
        ratings_url = '/contents/movies/ratings'
        user_movie = user_page+ratings_url
        driver.get(user_movie)
        time.sleep(2)

        # 광고 지우기
        close_button = '/html/body/div/div/div[2]/div/div/div/button/span'
        driver.find_element_by_xpath(close_button).click()

        # 스크롤 내리기
        while True:
            lastHeight = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            newHeight = driver.execute_script("return document.body.scrollHeight")
            if newHeight == lastHeight:
                 break

        # Beautifulsoup으로 영화 데이터
        movie_name = []
        movie_url = driver.current_url
        movie_link = []
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        movies = soup.find_all('li',{'class':'css-8y23cj'})

        for movie in movies:
            movie_name.append(movie.find('a').get('title'))
            movie_link.append(movie_url + movie.find('a').get('href'))

        df_movie_2 = pd.DataFrame(data = [movie_name, movie_link]).T

        # 위 데이터 프레임 행 = 그 페이지에 있는 영화 개수
        num = df_movie_2.shape[0]

        # Beautifulsoup으로 영화 데이터

        movie_name = []
        movie_link = []
        movie_score = []

        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        movies = soup.find_all('li',{'class':'css-8y23cj'})

        for movie in movies:
            movie_name.append(movie.find('a').get('title'))
            movie_link.append(movie_url + movie.find('a').get('href'))
        for k in range (1,num):
            item_score = driver.find_element_by_xpath(f'/html/body/div[1]/div/div[1]/section/section/div[1]/section/div[1]/div/ul/li[{k}]/a/div[2]/div[2]')
            movie_score.append(item_score.text)

        df_movie_2 = pd.DataFrame(data = [movie_name, movie_link,movie_score]).T

        data_df_new = pd.DataFrame(zip(movie_name,movie_link,movie_score),columns=['제목','링크','평점'])
        data_df_new['유저 이름'] = df_user.iloc[j]['유저']
        data_df_new
        
        data_df_users = data_df_users.append(data_df_new)
        data_df_users
        
    data_df = data_df.append(data_df_users)
    data_df

# 드라이버 끝내기
driver.close()
driver.quit()
    
data_df

Empty DataFrame
Columns: [제목, 링크, 평점, 유저 이름]
Index: []
